In [1]:
!nvidia-smi

Tue Apr 12 06:43:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    35W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -U transformers

     |████████████████████████████████| 4.0 MB 4.4 MB/s 
     |████████████████████████████████| 6.5 MB 36.4 MB/s 
     |████████████████████████████████| 596 kB 41.8 MB/s 
     |████████████████████████████████| 77 kB 6.0 MB/s 
     |████████████████████████████████| 895 kB 38.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [8]:
import pandas as pd
data = pd.read_csv('GVCEH-2022-04-11-tweet-raw.txt', index_col=0)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 264 entries, 0 to 263
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   text                  264 non-null    object
 1   scrape_time           264 non-null    object
 2   tweet_id              264 non-null    int64 
 3   created_at            264 non-null    object
 4   reply_count           264 non-null    int64 
 5   quote_count           264 non-null    int64 
 6   like_count            264 non-null    int64 
 7   retweet_count         264 non-null    int64 
 8   username              264 non-null    object
 9   num_followers         264 non-null    int64 
 10  search_keywords       264 non-null    object
 11  search_neighbourhood  264 non-null    object
dtypes: int64(6), object(6)
memory usage: 26.8+ KB


In [9]:
data.head()

,text,scrape_time,tweet_id,created_at,reply_count,quote_count,like_count,retweet_count,username,num_followers,search_keywords,search_neighbourhood
0,Hey @JustinTrudeau instead of talking to Lisa ...,2022-04-11 15:43:08.668693,1513580766441066498,2022-04-11 18:12:30+00:00,0,0,1,0,larknation69,13117,Victoria Homeless,Victoria
1,@SpongebobBirth @Flattielover @LadyOfTheOcean1...,2022-04-11 15:43:08.668693,1513574164057280512,2022-04-11 17:46:16+00:00,1,0,2,0,jerbear714,895,Victoria Homeless,Victoria
2,@KsLiberal @gopisdirty @TommyD07868371 @LadyOf...,2022-04-11 15:43:08.668693,1513298762009124864,2022-04-10 23:31:55+00:00,1,0,1,0,HatedByBCgovt,20,Victoria Homeless,Victoria
3,@HomeForHope @BurnsideGorge I am #homeowner #w...,2022-04-11 15:43:08.668693,1513216985059913730,2022-04-10 18:06:58+00:00,0,0,0,0,brightsider123,12509,Victoria Homeless,Victoria
4,Had loads of yellow sticker mushrooms and a ye...,2022-04-11 15:43:08.668693,1513201687476981762,2022-04-10 17:06:10+00:00,1,0,9,0,EndZonerBoner,259,Victoria Homeless,Victoria


In [10]:
from transformers import pipeline
model = pipeline("sentiment-analysis", model='cardiffnlp/twitter-roberta-base-sentiment-latest', device=0)

Downloading:   0%|          | 0.00/929 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [18]:
model.tokenizer.model_max_length = 512

In [20]:
model('I love GVCEH!')

[{'label': 'Positive', 'score': 0.9876995086669922}]

In [19]:
all_res = []
for res in model(data.text.to_list(), batch_size=32, truncation=True):
  all_res.append(res)
len(all_res)

264

In [23]:
all_sentiments = [x['label'] for x in all_res]
all_scores = [x['score'] for x in all_res]

In [24]:
data.text.iloc[5], all_sentiments[5], all_scores[5]

('@george_pattons @gopisdirty @PeterBartholo14 @DaniTrump2020 @Randomwatcher51 @Bad_Boy_Jesus @Mountain_Buddy @jerbear714 @Vickie627 @LadyOfTheOcean1 @fukit32 @Bigteethyouhave @PridgeWessea @Flattielover @akuscg @Synyster63 @KsLiberal @Frdmlvr3691 @TommyD07868371 @Saggit95 @donnnie @ShellyRKirchoff @PalusoSonny @SpongebobBirth @TheDaveWeinbaum @BC_Missy @victoria_roark @NathanBronson7 @Silver_Strike @skeeduu @NatForTrump2024 @Jetson77 @PhilipEMSkinne1 @newsjunky2 @Kittie_Svengali @SaveThe_Ukraine @sd_usn @JoeBro20802650 @CunningMarshal @SuP3rK @NanetteDonnelly @VinceGottalotta @mmEyeNewMe @chriscr66024638 @RushvilleRockit @Sembrouthes @giddy_bunny @RonVarney3 @jeff77531528 @Emma34770971 Taxes\nCriminals released \nCrime wave\nHunter\nCovid deaths skyrocket\nHomeless population growing\nThe red tsunami',
 'Neutral',
 0.7201572060585022)

In [25]:
data['sentiment'] = all_sentiments
data['score'] = all_scores
data.head()

,text,scrape_time,tweet_id,created_at,reply_count,quote_count,like_count,retweet_count,username,num_followers,search_keywords,search_neighbourhood,sentiment,score
0,Hey @JustinTrudeau instead of talking to Lisa ...,2022-04-11 15:43:08.668693,1513580766441066498,2022-04-11 18:12:30+00:00,0,0,1,0,larknation69,13117,Victoria Homeless,Victoria,Negative,0.759578
1,@SpongebobBirth @Flattielover @LadyOfTheOcean1...,2022-04-11 15:43:08.668693,1513574164057280512,2022-04-11 17:46:16+00:00,1,0,2,0,jerbear714,895,Victoria Homeless,Victoria,Neutral,0.770365
2,@KsLiberal @gopisdirty @TommyD07868371 @LadyOf...,2022-04-11 15:43:08.668693,1513298762009124864,2022-04-10 23:31:55+00:00,1,0,1,0,HatedByBCgovt,20,Victoria Homeless,Victoria,Neutral,0.815953
3,@HomeForHope @BurnsideGorge I am #homeowner #w...,2022-04-11 15:43:08.668693,1513216985059913730,2022-04-10 18:06:58+00:00,0,0,0,0,brightsider123,12509,Victoria Homeless,Victoria,Negative,0.768724
4,Had loads of yellow sticker mushrooms and a ye...,2022-04-11 15:43:08.668693,1513201687476981762,2022-04-10 17:06:10+00:00,1,0,9,0,EndZonerBoner,259,Victoria Homeless,Victoria,Positive,0.514702


In [26]:
data.to_csv('GVCEH-2022-04-11-tweet-raw-sentiment.csv', index=False)